In [1]:
import random
import numpy as np

In [2]:
def h_prima(H: int, mensaje: str) -> str:
    
    # Se divide el h0 en 4 partes de la siguiente forma
    mascara = 0xFFFFFFFF
    a0 = (H & (mascara << 96)) >> 96
    b0 = (H & (mascara << 64)) >> 64
    c0 = (H & (mascara << 32)) >> 32
    d0 = H & mascara
    
    # Se especifica los shifts por ronda
    s = []
    s[0:15] =  [ 7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22 ]
    s[16:31] = [ 5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20 ]
    s[32:47] = [ 4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23 ]
    s[48:63] = [ 6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21 ]
      
    k = []
    for i in range(64):
        k.append(int(np.floor(2**32*abs(np.sin(i + 1)))) & mascara)
    
    # Se inicializan las variables
    A = a0
    B = b0
    C = c0
    D = d0
    
    # Se divide el mensaje en chunks de 32 bits
    M = []
    for g in range(0,len(mensaje),4):
        M.append(int.from_bytes(mensaje[g:g+4], byteorder='little'))
    
    for j in range(64):
        F,g = 0,0
        if 0 <= j <= 15:
            F = (B & C) | ((~ B) & D)
            g = j
        elif 16 <= j <= 31:
            F = (D & B) | ((~ D) & C)
            g = (5*j + 1) % 16
        elif 32 <= j <= 47:
            F = B ^ C ^ D
            g = (3*j + 5) % 16
        elif 48 <= j <= 63:
            F = C ^ (B | (~ D))
            g = (7*j) % 16
        
        F = (F + A + k[j] + M[g]) & mascara
        A = D
        D = C
        C = B
        B = (B + (F << s[j] | F >> (32-s[j])) & mascara) & mascara
        
    a0 = (a0 + A) & mascara
    b0 = (b0 + B) & mascara
    c0 = (c0 + C) & mascara
    d0 = (d0 + D) & mascara

    return a0 + (b0 << 32) + (c0 << 64) + (d0 << 96)

In [3]:
def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

In [4]:
def tuplaGrupo():
    definitiva = []
    with open("grupo.txt", "r") as file:
        lineas = file.readlines()
        for linea in lineas:
            definitiva.append(linea.strip())
    largo = " ".join(definitiva)
    ahora_si = largo.split(",")
    primera = ahora_si[0].split(" ")
    segunda = ahora_si[1].split(" ")
    tercera = ahora_si[2].split(" ")
    p = int("0x" + "".join(primera), 16)
    g = int("0x" + "".join(segunda), 16)
    q = int("0x" + "".join(tercera), 16)
    return p, g, q

In [5]:
def alg_ext_euclides(a: int, b: int) -> (int, int, int):
    # Argumentos :
    # a: int
    # b: int - a >= b 0 y a > 0
    # Retorna:
    # (int, int, int) - maximo comun divisor MCD(a, b) entre a y b,
    # numeros enteros s y t tales que MCD(a, b) = s*a + t*b
    s1 = 1
    t1 = 0
    s2 = 0
    t2 = 1
    resultado = a % b
    mcd = b
    while resultado != 0:
        s_aux = s1 - (a // b)*s2
        t_aux = t1 - (a // b)*t2
        s1 = s2
        t1 = t2
        s2 = s_aux
        t2 = t_aux
        a = b
        b = resultado
        mcd = resultado
        resultado = a % b
    return (mcd, s2, t2)

In [6]:
def inverso(a: int, n: int) -> int:
    # Argumentos:
    # a: int - a >= 1
    # n: int - n >= 2, a y n son primos relativos
    # Retorna:
    # int - inverso de a en modulo n
    mcd, s, t = alg_ext_euclides(a,n)
    if mcd == 1:
        while s < 0:
            s += n
        if 0 <= s <= n-1:
            return s

In [7]:
def exp_mod(a: int, b: int, n: int) -> int:
    # Argumentos:
    # a: int - a >= 0
    # b: int - b >= 0
    # n: int - n > 0
    # Retorna:
    # int - a**b en modulo n
    if b == 0:
        return 1
    elif b%2 == 0:
        t = exp_mod(a, b//2, n)
        return (t*t)%n
    else:
        t = exp_mod(a, (b-1)//2, n)
        return (t*t*a)%n

In [8]:
def md5(m: str) -> int:
    # Argumentos :
    # m: str - mensaje
    # Retorna :
    # int - valor de la funcion de hash MD5 aplicada sobre m
    message = bytearray(m, 'utf-8') #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message)%64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

        
    # Ahora se toman los estados para calcular el h_prima
    H = 137269462086865085541390238039692956790
    for i in range(0,len(message),64):
        H = h_prima(H, message[i: i + 64])
        
    return md5_to_hex(H)   

In [9]:
def generar_clave_ElGamal():
    # Retorna :
    # Genera una clave privada y una clave publica según el protocolo
    # criptográfico ElGamal, para el grupo alamacenado en grupo.txt.
    # Almacena la clave privada en private_key.txt, y la clave publica
    # en public_key.txt
    p, g, q = tuplaGrupo()
    private_key = random.randint(1, q - 1)
    public_key = exp_mod(g, private_key, p)
    with open("private_key.txt", "w") as private:
        private.write(str(private_key) +"\n")
    
    with open("public_key.txt", "w") as public:
        public.write(str(public_key) + "\n")

In [10]:
def firmar_Schnorr(m: str) -> (int, int):
    # Argumentos :
    # m: str - mensaje
    # Retorna :
    # (int, int) - firma de Schnorr (e, s) del mensaje m según la clave
    # privada almacenada en private_key.txt, para el grupo almacenado
    # en grupo.txt
    with open("private_key.txt") as private:
        privada = private.readline().strip()
    privada = int(privada)
    definitiva = []
    p, g, q = tuplaGrupo()
    k = random.randint(1, q - 1)
    r = exp_mod(g, k, p)
    e = md5(str(r) + m)
    s = (k - privada*int(e, 16))
    return (int(e, 16), s)

In [15]:
def verificar_firma_Schnorr(m: str, firma: (int, int)) -> bool:
    # Argumentos :
    # m: str - mensaje
    # firma: (int, int) - firma de Schnorr (e, s) para m
    # Retorna: 
    # bool - retorna True si para el usuario con clave publica
    # almacenada en public_key.txt, el par (e, s) es una firma de
    # Schnorr correcta para el mensaje m, en el grupo almacenado en
    # grupo.txt. En caso contrario retorna False
    p, g, q = tuplaGrupo()
    with open("public_key.txt", "r") as public:
        publica = public.readline().strip()
    publica = int(publica)
    if firma[1] < 0:
        inverso_mod = inverso(g, p)
        r_prima1 = exp_mod(inverso_mod, firma[1]*-1, p)
    else:
        r_prima1 = exp_mod(g, firma[1], p)
    r_prima2 = exp_mod(publica, firma[0], p)
    r_prima = exp_mod(r_prima1*r_prima2, 1, p)
    md5_output = md5(str(r_prima) + m)
    return int(md5_output, 16) == firma[0]

In [16]:
generar_clave_ElGamal()

In [19]:
firme = firmar_Schnorr("chaaao")
firme

(28624707906479628785782656608920168138,
 -955595366342964847313864264423390605639516746144042611561314882494630801523485131427896105982786589614862438672872)

In [21]:
verificar_firma_Schnorr("chaahola", firme)

False